In [1]:
# Importons les librairies dont nous avons besoin.
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-muted')
%matplotlib inline
import numpy as np
from sklearn import preprocessing
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import DistanceMetric
from sklearn.cluster import KMeans
import random
import math

# Modélisation

## Nettoyage complémentaire

Puisque dans ce modèle notre but est de trouver la ressemblance entre deux films nous allons dans un premier temps laisser certaines colonnes de côté. Ainsi, le nombre de votes et de likes qui synthétisent plus la popularité que la ressemblance entre deux films de seront pas utilisés tout de suite. 

Ces indicateurs pourraient nous être utiles en complèment de notre modèle et c'est pourquoi nous avons décider de les garder durant le nettoyagen mais pour la première version de notre modèle basé nous n'en avons pas besoin a priori.

Gardons ces colonnes de côté, ainsi que les autres colonnes de type object afin d'avoir du contenu à afficher à l'utilisateur, et créons un autre dataframe normalisé qui servira notre modèle.

In [2]:
global_data = pd.read_csv("clean_data.csv", header=0)
user_data = global_data[['movie_title', 'director_name', 'duration', 'title_year','country','genres',"actors", "colors", "plot_keywords", "imdb_score"]]
data = global_data.drop(columns=global_data.loc[:,"country":'plot_keywords'].columns).drop(columns=["actors", "colors", 'movie_facebook_likes', "movie_title", "director_name"])
std_scale = preprocessing.StandardScaler().fit(data)
X_scaled = std_scale.transform(data)
X_scaled = pd.DataFrame(X_scaled, columns = data.columns)
X_scaled.head()

,duration,title_year,imdb_score,genre_western,genre_thriller,genre_animation,genre_mystery,genre_music,genre_action,genre_comedy,...,summer,jason voorhe,found footag,killer,irish,independent film,satir,supernatur,oral sex,mormon
0,3.129545,0.528642,1.335324,-0.141708,-0.621666,-0.224441,-0.325718,-0.267561,1.837585,-0.778885,...,-0.071074,-0.045811,-0.050194,-0.056136,-0.057983,-0.064854,-0.054227,-0.072548,-0.045811,-0.045811
1,2.724770,0.367213,0.619333,-0.141708,-0.621666,-0.224441,-0.325718,-0.267561,1.837585,-0.778885,...,-0.071074,-0.045811,-0.050194,-0.056136,-0.057983,-0.064854,-0.054227,-0.072548,-0.045811,-0.045811
2,1.780295,1.012930,0.350836,-0.141708,1.608580,-0.224441,-0.325718,-0.267561,1.837585,-0.778885,...,-0.071074,-0.045811,-0.050194,-0.056136,-0.057983,-0.064854,-0.054227,-0.072548,-0.045811,-0.045811
3,2.499895,0.770786,1.872318,-0.141708,1.608580,-0.224441,-0.325718,-0.267561,1.837585,-0.778885,...,-0.071074,-0.045811,-0.050194,-0.056136,-0.057983,-0.064854,-0.054227,-0.072548,-0.045811,-0.045811
4,1.060694,0.770786,0.171838,-0.141708,-0.621666,-0.224441,-0.325718,-0.267561,1.837585,-0.778885,...,-0.071074,-0.045811,-0.050194,-0.056136,-0.057983,-0.064854,-0.054227,-0.072548,-0.045811,-0.045811


## KNN

Commençons par modéliser notre problème en reprennant l'idée du KNN. Pour cela il suffit de construire une matrice de distance entre tous les films deux à deux, et de sélectionner ensuite les k films les plus proches.

Plusieurs notions de distances sont possibles, analysons-en deux : Euclidienne et de Manhattan. Afin de vérifier leur pertinence, regardons les films proposés pour Batman v Superman: Dawn of Justice, Pirates of the Caribbean: At World's End, Quantum of Solace et Titanic. Nous sélectionnerons les 5 films les plus proches pour chacun d'entre-eux.

### Eculidean distance

In [3]:
dist = DistanceMetric.get_metric('euclidean')
X = dist.pairwise(X_scaled)

In [4]:
user_data.loc[list(np.argsort(X[1])[:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
1,Pirates of the Caribbean: At World's End,Gore Verbinski,169.0,2007.0,USA,Action|Adventure|Fantasy,Johnny Depp|Orlando Bloom|Jack Davenport,Color,goddess|marriage ceremoni|marriage propos|pira...,7.1
200,Pirates of the Caribbean: The Curse of the Bla...,Gore Verbinski,143.0,2003.0,USA,Action|Adventure|Fantasy,Johnny Depp|Orlando Bloom|Jack Davenport,Color,caribbean|curs|governor|pirat|undead,8.1
12,Pirates of the Caribbean: Dead Man's Chest,Gore Verbinski,151.0,2006.0,USA,Action|Adventure|Fantasy,Johnny Depp|Orlando Bloom|Jack Davenport,Color,box office hit|giant squid|heart|liar's dic|mo...,7.3
2589,The Pirates Who Don't Do Anything: A VeggieTal...,Mike Nawrocki,85.0,2008.0,USA,Adventure|Animation|Comedy|Family,Yuri Lowenthal|Cam Clarke|Phil Vischer,Color,17th centuri|king|pirat|veget|waiter,5.7
2679,Jonah: A VeggieTales Movie,Mike Nawrocki,82.0,2002.0,USA,Adventure|Animation|Comedy|Drama|Family|Musical,Phil Vischer|Mike Nawrocki|Shelby Vischer,Color,anthropomorph|pirat|pirate ship|tomato|whale,6.6


In [5]:
user_data.loc[list(np.argsort(X[9])[:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
9,Batman v Superman: Dawn of Justice,Zack Snyder,183.0,2016.0,USA,Action|Adventure|Sci-Fi,Henry Cavill|Lauren Cohan|Alan D. Purwin,Color,based on comic book|batman|sequel to a reboot|...,6.9
539,Immortals,Tarsem Singh,110.0,2011.0,USA,Action|Drama|Fantasy|Romance,Henry Cavill|Daniel Sharman|Steve Byers,Color,burned al|castrat|olympus|tortur|torture devic,6.1
2244,The Cold Light of Day,Mabrouk El Mechri,93.0,2012.0,USA,Action|Thriller,Henry Cavill|Bruce Willis|Óscar Jaenada,Color,driving a car down stair|duct tape gag|held ca...,4.9
14,Man of Steel,Zack Snyder,143.0,2013.0,USA,Action|Adventure|Fantasy|Sci-Fi,Henry Cavill|Christopher Meloni|Harry Lennix,Color,based on comic book|british actor playing amer...,7.2
4544,The Toxic Avenger,Michael Herz,91.0,1984.0,USA,Action|Comedy|Horror|Sci-Fi,Patrick Kilpatrick|Mark Torgl|Pat Ryan,Color,boy|health club|superhero|toxic wast|troma,6.2


In [6]:
user_data.loc[list(np.argsort(X[11])[:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
11,Quantum of Solace,Marc Forster,106.0,2008.0,UK,Action|Adventure,Giancarlo Giannini|Mathieu Amalric|Rory Kinnear,Color,action hero|attempted rap|bond girl|official j...,6.7
530,Munich,Steven Spielberg,163.0,2005.0,France,Drama|History|Thriller,Ayelet Zurer|Moritz Bleibtreu|Mathieu Amalric,Color,arab|black septemb|israel|jew|munich olymp,7.6
2186,The Kite Runner,Marc Forster,128.0,2007.0,USA,Drama,Mustafa Haidari|Shaun Toub|Khalid Abdalla,Color,afghanistan|based on novel|boy|friend|kite,7.6
4249,The Blue Room,Mathieu Amalric,76.0,2014.0,France,Crime|Romance|Thriller,Mathieu Amalric|Léa Drucker|Laurent Poitrenaux,Color,male pubic hair|manipulative person|murder of ...,6.3
2698,The Diving Bell and the Butterfly,Julian Schnabel,112.0,2007.0,France,Biography|Drama,Emmanuelle Seigner|Mathieu Amalric|Isaach De B...,Color,arc de triomphe pari|based on autobiographi|co...,8.0


In [7]:
user_data.loc[list(np.argsort(X[25])[:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
25,Titanic,James Cameron,194.0,1997.0,USA,Drama|Romance,Leonardo DiCaprio|Kate Winslet|Gloria Stuart,Color,artist|love|ship|titan|wet,7.7
4723,The Brothers McMullen,Edward Burns,98.0,1995.0,USA,Comedy|Drama|Romance,Shari Albert|Michael McGlone|Maxine Bahns,Color,abusive fath|critically acclaim|loss of fath|l...,6.6
3296,Mr. Church,Bruce Beresford,104.0,2016.0,USA,Drama,Mckenna Grace|Lucy Fry|Natalie Coughlin,Color,NaN,8.0
4663,The Man from Earth,Richard Schenkman,87.0,2007.0,USA,Drama|Romance|Sci-Fi,William Katt|John Billingsley|David Lee Smith,Color,cro magnon|dialogue driven storylin|single set...,8.0
3466,City Island,Raymond De Felitta,104.0,2009.0,USA,Comedy|Drama,Ezra Miller|Curtiss Cook|Dominik García-Lorido,Color,anger|family reunion|father son reunion|hug|mo...,7.4


### Manhattan distance

In [8]:
dist = DistanceMetric.get_metric('manhattan')
X = dist.pairwise(X_scaled)

In [9]:
user_data.loc[list(np.argsort(X[1])[:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
1,Pirates of the Caribbean: At World's End,Gore Verbinski,169.0,2007.0,USA,Action|Adventure|Fantasy,Johnny Depp|Orlando Bloom|Jack Davenport,Color,goddess|marriage ceremoni|marriage propos|pira...,7.1
200,Pirates of the Caribbean: The Curse of the Bla...,Gore Verbinski,143.0,2003.0,USA,Action|Adventure|Fantasy,Johnny Depp|Orlando Bloom|Jack Davenport,Color,caribbean|curs|governor|pirat|undead,8.1
12,Pirates of the Caribbean: Dead Man's Chest,Gore Verbinski,151.0,2006.0,USA,Action|Adventure|Fantasy,Johnny Depp|Orlando Bloom|Jack Davenport,Color,box office hit|giant squid|heart|liar's dic|mo...,7.3
3296,Mr. Church,Bruce Beresford,104.0,2016.0,USA,Drama,Mckenna Grace|Lucy Fry|Natalie Coughlin,Color,NaN,8.0
3244,Kicks,Justin Tipping,80.0,2016.0,USA,Adventure,Tina Gilton|Natalie Stephany Aguilar|Justin Hall,Color,NaN,7.8


In [10]:
user_data.loc[list(np.argsort(X[9])[:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
9,Batman v Superman: Dawn of Justice,Zack Snyder,183.0,2016.0,USA,Action|Adventure|Sci-Fi,Henry Cavill|Lauren Cohan|Alan D. Purwin,Color,based on comic book|batman|sequel to a reboot|...,6.9
243,Teenage Mutant Ninja Turtles: Out of the Shadows,Dave Green,112.0,2016.0,USA,Action|Adventure|Comedy|Sci-Fi,Stephen Amell|Noel Fisher|Brad Garrett,Color,based on comic book|brother brother relationsh...,6.3
4544,The Toxic Avenger,Michael Herz,91.0,1984.0,USA,Action|Comedy|Horror|Sci-Fi,Patrick Kilpatrick|Mark Torgl|Pat Ryan,Color,boy|health club|superhero|toxic wast|troma,6.2
14,Man of Steel,Zack Snyder,143.0,2013.0,USA,Action|Adventure|Fantasy|Sci-Fi,Henry Cavill|Christopher Meloni|Harry Lennix,Color,based on comic book|british actor playing amer...,7.2
26,Captain America: Civil War,Anthony Russo,147.0,2016.0,USA,Action|Adventure|Sci-Fi,Robert Downey Jr.|Scarlett Johansson|Chris Evans,Color,based on comic book|knife|marvel cinematic uni...,8.2


In [11]:
user_data.loc[list(np.argsort(X[11])[:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
11,Quantum of Solace,Marc Forster,106.0,2008.0,UK,Action|Adventure,Giancarlo Giannini|Mathieu Amalric|Rory Kinnear,Color,action hero|attempted rap|bond girl|official j...,6.7
935,Shanghai Knights,David Dobkin,114.0,2003.0,USA,Action|Adventure|Comedy,Fann Wong|Anna-Louise Plowman|Georgina Chapman,Color,1880s|arrow|imperial s|murder|reveng,6.2
4227,Road Hard,Adam Carolla,98.0,2015.0,USA,Comedy,Jay Mohr|Jim O'Heir|David Alan Grier,Color,girl in panti|panti|red panti|stand up comedi,6.1
2186,The Kite Runner,Marc Forster,128.0,2007.0,USA,Drama,Mustafa Haidari|Shaun Toub|Khalid Abdalla,Color,afghanistan|based on novel|boy|friend|kite,7.6
4674,The World Is Mine,Nicolae Constantin Tanase,104.0,2015.0,Romania,Drama,Iulia Ciochina|Ana Maria Guran|Ana Vatamanu,Color,NaN,6.5


In [12]:
user_data.loc[list(np.argsort(X[25])[:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
25,Titanic,James Cameron,194.0,1997.0,USA,Drama|Romance,Leonardo DiCaprio|Kate Winslet|Gloria Stuart,Color,artist|love|ship|titan|wet,7.7
4723,The Brothers McMullen,Edward Burns,98.0,1995.0,USA,Comedy|Drama|Romance,Shari Albert|Michael McGlone|Maxine Bahns,Color,abusive fath|critically acclaim|loss of fath|l...,6.6
4663,The Man from Earth,Richard Schenkman,87.0,2007.0,USA,Drama|Romance|Sci-Fi,William Katt|John Billingsley|David Lee Smith,Color,cro magnon|dialogue driven storylin|single set...,8.0
3296,Mr. Church,Bruce Beresford,104.0,2016.0,USA,Drama,Mckenna Grace|Lucy Fry|Natalie Coughlin,Color,NaN,8.0
3466,City Island,Raymond De Felitta,104.0,2009.0,USA,Comedy|Drama,Ezra Miller|Curtiss Cook|Dominik García-Lorido,Color,anger|family reunion|father son reunion|hug|mo...,7.4


### Analyse des résultats pour les distances

Concernant Pirates des Caraïbes on voit que le choix ne varie que pour les 4e et 5e films. La distance Euclidienne nous propose The Pirates Who Don't Do Anything: A VeggieTal...	quand la distance de Manhattan nous propose Out of Mr Church et Kicks. 
Ces recommandations ne sont pas forcément très bonnes, difficile de départager les distances, totuefois la distance euclidienne a retenu l'idée de pirate et semble donc légérement plus logique.

Concernant Batman les choix différent pour deux films :
La distance Euclidienne nous propose The Cold Light of Day et Immortals alors que la distance de Manhattan nous propose Teenage Mutant Ninja Turtles: Out of the Shadows et Captain America: Civil War.
Les deux recommandations paraissent intéressantes, difficile de les départager.


Pour James Bond : The Kite Runner, Munich, The Blue Room  et The Diving Bell and the Butterfly pour la distance Euclidienne. Shanghai Knights, Road Hard, The Kite Runner et The World Is Mine pour la distance de Manhattan. Le dernier film est commun. La distance euclidienne semble plus appropriée ici et l'on voit bien qu'elle recommande des films où jouent Mathieu Amalric. Il est en revanche plus difficile de comprendre le choix de la distance de Manhattan.

Pour Titani les prédictions sont les mêmes.

Globalement les résultats sont proches mais s'il fallait en choisir une nous retiendrons la distance Euclidienne qui semble nous satisfaire un peu plus notamment pour le film Quantum of Solace.

### Limite de la distance de Manhattan

Comme mis en avant avec le cas de Quantum of Solace, notre algorithme en version de Manhattan est en difficulé lorsqu'il doit recommander des films avec peu d'acteurs ou de mots clefs référence (Mathieu Amalric uniquement dans notre cas). Illustrons cela avec la différence chiffrée de Quantum of Solace et Road Hard et de Quantum of Solace et Tomorrow Never Dies par exemple.

In [13]:
diff = pd.DataFrame(X_scaled.iloc[11] - X_scaled.iloc[4227])
diff[diff[0] != 0]

,0
duration,0.359800
title_year,-0.565002
imdb_score,0.536994
genre_action,2.381778
genre_comedy,-2.062771
genre_adventure,2.588307
country_uk,3.538274
country_usa,-2.329752
director_name_marc forster,24.451517
pg-13,2.191253


In [14]:
diff = pd.DataFrame(X_scaled.iloc[11] - X_scaled.iloc[247])
diff[diff[0] != 0]

,0
duration,-0.584675
title_year,0.887860
imdb_score,0.178998
genre_thriller,-2.230246
director_name_marc forster,24.451517
vincent_schiavelli,-30.919267
colin_salmon,-23.055530
mathieu_amalric,30.919267
joe_don_baker,-30.919267
reveng,8.320406


On se rend compte ici que Tomorrow Never Dies est pénalisé du fait que Colin Salmon et Vincent Schiavelli y jouent et non dans Quantum of Solace. De la même façon Skyfall sera pénalisé parce que le directeur est Sam Mendes. 

Ici, il est donc dommage de remarquer que la présence d'acteurs ou de directeurs pénalise notre algorithme en comparaison à des films dont aucun acteur n'est reconnu. 

Ainsi, notre algorithme considère qu'un film avec Colin Salmon est plus distant qu'un film avec Lhakpa Dorji d'un film ou aucun de ces deux acteurs ne joue. De ce fait, les films avec peu d'acteurs ou de directeurs connus seront considérés comme loin entre eux si tôt fait que le seul acteur connu qui joue ne soit pas commun.

Si cet effet est potentiellement présent dans la distance Euclidienne, il était en tout cas moins visible.

### Tirage changeant

On remarque également que l'algorithme nous propose logiquement les différents épisodes d'un film lorsqu'il en comporte. Ainsi, il n'est pas très pertinent de proposer tous les épisodes d'une même série comme pour Pirates des Caraibes. 

De plus, afin de proposer un contenu plus varié à nos utilisateurs, il serait intéressant de ne pas toujours proposer la même liste. Nous pourrions donc prendre les 15 plus proches voisins et en tirer aléatoirement 5 à chaque fois afin de proposer des tirages différents et de ne pas se limiter aux différents épisodes d'une suite.

## K-means

Après avoir travaillé sur un KNN utilons désormais un algorithme de clustering qu'est le K-means et qui pourrait nous proposer des résultats différents.

Sachant que nous voudrions 5 films par cluster globalement, essayons de définir 1000 clusters dans un premier temps et commençons par analyser nos résultats avec Pirates des Caraibes.

In [15]:
kmeans = KMeans(n_clusters=1000, random_state=0).fit(X_scaled)

In [16]:
indexes = [i for i,x in enumerate(list(kmeans.labels_)) if x == kmeans.labels_[1]]
indexes

[1, 12, 200]

On voit ici que pour Pirates des Caraibes seuls deux autres films sont recommandés : deux autres épisodes, ce qui ne correpond pas à nos besoins. Pour pallier cela, essayons cette fois de réduire de moitié le nombre de clusters pour voir si leur taille augmente.

In [17]:
kmeans = KMeans(n_clusters=500, random_state=0).fit(X_scaled)

In [18]:
indexes = [i for i,x in enumerate(list(kmeans.labels_)) if x == kmeans.labels_[1]]
indexes

[1, 12, 200]

Le même problème se retrouve ici... Essayons de réduire jusqu'à 200 clusters.

In [19]:
kmeans = KMeans(n_clusters=200, random_state=0).fit(X_scaled)

In [20]:
indexes = [i for i,x in enumerate(list(kmeans.labels_)) if x == kmeans.labels_[1]]
len(indexes)

2859

A l'inverse, cette fois notre cluster est bien trop grand...

On peut donc éliminer l'agorithme K-means qui ne permettra pas dans sa forme initiale de répondre à notre problème. Une solution serait de forcer la taille des clusters, mais cela reviendrait à modifier le K-means et à nous ramener à un KNN. Les algoritmhes de clustering sont intéressants lorsqu'il s'agit de regrouper ou de catégoriser une population, mais pas lorsque celle-ci doit être groupée en des proportions similaires (en ce qui concerne le K-means en tout cas).

### KNN version 2

Essayons de voir si nous pouvons améliorer nos résultats en reprennant notre KNN avec distance Euclidienne qui est l'algorithme ayant le mieux performé jusque là. Plutôt que de normaliser nos différentes colonnes, nous ne le ferons que pour celles qui ne concernent pas le one hot encoding.

In [21]:
global_data = pd.read_csv("clean_data.csv", header=0)
user_data = global_data[['movie_title', 'director_name', 'duration', 'title_year','country','genres',"actors", "colors", "plot_keywords", "imdb_score"]]

data = global_data.drop(columns=global_data.loc[:,"movie_title":'movie_facebook_likes'].columns).drop(columns=["actors", "colors"])
std_scale = preprocessing.StandardScaler().fit(global_data.loc[:,["duration", "title_year", "imdb_score"]])
X_scaled = std_scale.transform(global_data.loc[:,["duration", "title_year", "imdb_score"]])
X_scaled = pd.DataFrame(X_scaled, columns = ["duration", "title_year", "imdb_score"])
X_scaled = X_scaled.merge(data, left_index=True, right_index=True, how='outer')
X_scaled.head()

,duration,title_year,imdb_score,genre_western,genre_thriller,genre_animation,genre_mystery,genre_music,genre_action,genre_comedy,...,summer,jason voorhe,found footag,killer,irish,independent film,satir,supernatur,oral sex,mormon
0,3.129545,0.528642,1.335324,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2.724770,0.367213,0.619333,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1.780295,1.012930,0.350836,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2.499895,0.770786,1.872318,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1.060694,0.770786,0.171838,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### Euclidienne distance

In [22]:
dist = DistanceMetric.get_metric('euclidean')
X = dist.pairwise(X_scaled)

In [23]:
user_data.loc[list(np.argsort(X[1])[:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
1,Pirates of the Caribbean: At World's End,Gore Verbinski,169.0,2007.0,USA,Action|Adventure|Fantasy,Johnny Depp|Orlando Bloom|Jack Davenport,Color,goddess|marriage ceremoni|marriage propos|pira...,7.1
200,Pirates of the Caribbean: The Curse of the Bla...,Gore Verbinski,143.0,2003.0,USA,Action|Adventure|Fantasy,Johnny Depp|Orlando Bloom|Jack Davenport,Color,caribbean|curs|governor|pirat|undead,8.1
12,Pirates of the Caribbean: Dead Man's Chest,Gore Verbinski,151.0,2006.0,USA,Action|Adventure|Fantasy,Johnny Depp|Orlando Bloom|Jack Davenport,Color,box office hit|giant squid|heart|liar's dic|mo...,7.3
20,The Amazing Spider-Man,Marc Webb,153.0,2012.0,USA,Action|Adventure|Fantasy,Emma Stone|Andrew Garfield|Chris Zylka,Color,lizard|outcast|spider|spider man|teenag,7.0
98,The Hobbit: An Unexpected Journey,Peter Jackson,182.0,2012.0,USA,Adventure|Fantasy,Aidan Turner|Adam Brown|James Nesbitt,Color,dragon|dwarf|hobbit|orc|wizard,7.9


In [24]:
user_data.loc[list(np.argsort(X[9])[:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
9,Batman v Superman: Dawn of Justice,Zack Snyder,183.0,2016.0,USA,Action|Adventure|Sci-Fi,Henry Cavill|Lauren Cohan|Alan D. Purwin,Color,based on comic book|batman|sequel to a reboot|...,6.9
26,Captain America: Civil War,Anthony Russo,147.0,2016.0,USA,Action|Adventure|Sci-Fi,Robert Downey Jr.|Scarlett Johansson|Chris Evans,Color,based on comic book|knife|marvel cinematic uni...,8.2
36,Transformers: Age of Extinction,Michael Bay,165.0,2014.0,USA,Action|Adventure|Sci-Fi,Bingbing Li|Sophia Myles|Kelsey Grammer,Color,blockbust|bumblebee the charact|semi truck and...,5.7
14,Man of Steel,Zack Snyder,143.0,2013.0,USA,Action|Adventure|Fantasy|Sci-Fi,Henry Cavill|Christopher Meloni|Harry Lennix,Color,based on comic book|british actor playing amer...,7.2
64,X-Men: Apocalypse,Bryan Singer,144.0,2016.0,USA,Action|Adventure|Sci-Fi,Jennifer Lawrence|Michael Fassbender|Tye Sheridan,Color,mutant|superhero|superhero team|x men|year 1983,7.3


In [25]:
user_data.loc[list(np.argsort(X[11])[:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
11,Quantum of Solace,Marc Forster,106.0,2008.0,UK,Action|Adventure,Giancarlo Giannini|Mathieu Amalric|Rory Kinnear,Color,action hero|attempted rap|bond girl|official j...,6.7
935,Shanghai Knights,David Dobkin,114.0,2003.0,USA,Action|Adventure|Comedy,Fann Wong|Anna-Louise Plowman|Georgina Chapman,Color,1880s|arrow|imperial s|murder|reveng,6.2
2309,Out of Inferno,Danny Pang,107.0,2013.0,China,Action,Louis Koo|Angelica Lee|Ching Wan Lau,Color,NaN,5.7
4470,Starsuckers,Chris Atkins,103.0,2009.0,UK,Documentary,Richard Curtis|Harvey Weinstein|Elton John,Color,celebr|consumer|fashion|media manipul,7.4
3989,Silver Medallist,Hao Ning,99.0,2009.0,China,Action|Adventure|Comedy,Jack Kao|Bo Huang|Zheng Xu,Color,color in titl,7.4


In [26]:
user_data.loc[list(np.argsort(X[25])[:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
25,Titanic,James Cameron,194.0,1997.0,USA,Drama|Romance,Leonardo DiCaprio|Kate Winslet|Gloria Stuart,Color,artist|love|ship|titan|wet,7.7
1335,Magnolia,Paul Thomas Anderson,188.0,1999.0,USA,Drama,Patton Oswalt|Neil Flynn|Jim Meskimen,Color,abusive fath|coincid|fictional game show|ficti...,8.0
558,Braveheart,Mel Gibson,178.0,1995.0,USA,Biography|Drama|History|War,Mhairi Calvey|Patrick McGoohan|James Robinson,Color,14th centuri|legend|revolt|scotland|tyranni,8.4
100,The Curious Case of Benjamin Button,David Fincher,166.0,2008.0,USA,Drama|Fantasy|Romance,Brad Pitt|Jason Flemyng|Julia Ormond,Color,deformed babi|diari|lingerie slip|older man yo...,7.8
589,The Abyss,James Cameron,171.0,1989.0,USA,Adventure|Drama|Sci-Fi|Thriller,Michael Biehn|Todd Graff|Mary Elizabeth Mastra...,Color,alien|estranged coupl|ocean|submarin|underwat,7.6


Si cette approche évite de pénaliser fortement la présence d'autres acteurs connus que ceux présents dans le film recherché, elle minimise la présence d'acteurs communs et n'est donc pas très satsfaisantes... On voit ainsi que nos suggestions pour Quantum of Solace sont moins bonnes qu'elles ne l'étaient auparavant. En revanche les propositions pour Superman sont légérement meilleures. Ici il y a une part de subjectivité et nous ferons le choix de privilégier la présence d'acteurs communs plutôt que le genre du film et c'est pourquoi cette technique ne sera pas retenue.

Avant de continuer de modifier notre modèle, nous pouvons déjà créer la fonction nous permettant de tirer aléatoirement 5 films parmi les 15 plus proches voisins.

In [27]:
def random_neighbours(movie_index):
    movie = X[movie_index]
    best_neighbours = list(np.argsort(movie)[:16])
    del best_neighbours[0]
    random_neighbours = random.sample(best_neighbours, 5)
    return user_data.loc[random_neighbours,:]

Regardons ce que cela donnerait avec Quantom of Solace.

In [28]:
random_neighbours(11)

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
4434,Crying with Laughter,Justin Molotnikov,93.0,2009.0,UK,Thriller,Jo Hartley|Stephen McCole|Joe Cassidy,Color,cocain|edinburgh scotland|stand up comedian,6.9
3711,Falcon Rising,Ernie Barbarash,103.0,2014.0,USA,Action|Adventure,Michael Jai White|Michael J. Morris|Lateef Cro...,Color,NaN,5.8
4483,They Will Have to Kill Us First,Johanna Schwartz,105.0,2015.0,UK,Documentary,Aliou Touré|Garba Touré|Khaira Arby,Color,NaN,7.7
2309,Out of Inferno,Danny Pang,107.0,2013.0,China,Action,Louis Koo|Angelica Lee|Ching Wan Lau,Color,NaN,5.7
4124,"To Be Frank, Sinatra at 100",Simon Napier-Bell,81.0,2015.0,UK,Documentary,Alice Cooper|Tim Rice|Louis Walsh,Color,NaN,7.4


### KNN version 3

Essayons d'utiliser certaines colonnes que nous avions laissées de côté jusqu'alors et qui pourraient nous aider à retranscrire la popularité d'un film ce qui n'est jusqu'alors pas forcément visible lorsqu'aucun acteur vedette n'est présent.

Ajoutons donc trois colonnes pour notre analyse, le nombre de likes total du casting, du film ainsi que le nombre de personnes lui ayant attribué une note.

Enfin, ajoutons également une modification notable à notre algorithme. Afin de ne pas pénaliser les films comportants des acteurs connus et non présents dans le film recherché nous allons modifier notre algorithme. Plutôt que de nous intéresser à la distance des films, nous allons nous intéresser à leur ressemblance. 

Si je demande des suggestions pour Quantum of Solace, l'algorithme rapprochera les films dans lesquels figurent Mathieu Amalric mais n'éloignera pas les films où Di Caprio joue.

Pour cela nous allons donc utiliser notre propre définition de la distance et modifiant le calcul pour les acteurs et directeurs et en ne prenant en compte que leur ressemblance.

In [29]:
global_data = pd.read_csv("clean_data.csv", header=0)
user_data = global_data[['movie_title', 'director_name', 'duration', 'title_year','country','genres',"actors", "colors", "plot_keywords", "imdb_score"]]
data = global_data.drop(columns=global_data.loc[:,"country":'plot_keywords'].columns).drop(columns=["actors", "colors", 'movie_facebook_likes', "movie_title", "director_name"])
std_scale = preprocessing.StandardScaler().fit(data)
X_scaled = std_scale.transform(data)
X_scaled = pd.DataFrame(X_scaled, columns = data.columns)
X_scaled.head()

,duration,title_year,imdb_score,genre_western,genre_thriller,genre_animation,genre_mystery,genre_music,genre_action,genre_comedy,...,summer,jason voorhe,found footag,killer,irish,independent film,satir,supernatur,oral sex,mormon
0,3.129545,0.528642,1.335324,-0.141708,-0.621666,-0.224441,-0.325718,-0.267561,1.837585,-0.778885,...,-0.071074,-0.045811,-0.050194,-0.056136,-0.057983,-0.064854,-0.054227,-0.072548,-0.045811,-0.045811
1,2.724770,0.367213,0.619333,-0.141708,-0.621666,-0.224441,-0.325718,-0.267561,1.837585,-0.778885,...,-0.071074,-0.045811,-0.050194,-0.056136,-0.057983,-0.064854,-0.054227,-0.072548,-0.045811,-0.045811
2,1.780295,1.012930,0.350836,-0.141708,1.608580,-0.224441,-0.325718,-0.267561,1.837585,-0.778885,...,-0.071074,-0.045811,-0.050194,-0.056136,-0.057983,-0.064854,-0.054227,-0.072548,-0.045811,-0.045811
3,2.499895,0.770786,1.872318,-0.141708,1.608580,-0.224441,-0.325718,-0.267561,1.837585,-0.778885,...,-0.071074,-0.045811,-0.050194,-0.056136,-0.057983,-0.064854,-0.054227,-0.072548,-0.045811,-0.045811
4,1.060694,0.770786,0.171838,-0.141708,-0.621666,-0.224441,-0.325718,-0.267561,1.837585,-0.778885,...,-0.071074,-0.045811,-0.050194,-0.056136,-0.057983,-0.064854,-0.054227,-0.072548,-0.045811,-0.045811


In [30]:
def distance(x, y):
    x_actors = x[50:976]
    y_actors = y[50:976]
    z_actors = x_actors[(x_actors-y_actors == 0) & (x_actors > 0)]
    x = np.concatenate((x[:50], x[976:]), axis=0)
    y = np.concatenate((y[:50], y[976:]), axis=0)    
    z = (x - y)[x - y != 0]
    return sum(z_actors) - math.sqrt(sum((z)**2))

In [31]:
dist = DistanceMetric.get_metric('pyfunc', func=distance)
X = dist.pairwise(X_scaled)

Il nous faut modifier notre fonction random afin qu'elle prend les plus proches et non plus les plus distants.

In [32]:
def random_neighbours(movie_index):
    movie = X[movie_index]
    best_neighbours = list(np.argsort(movie)[::-1][:16])
    del best_neighbours[0]
    random_neighbours = random.sample(best_neighbours, 5)
    return user_data.loc[random_neighbours,:]

In [33]:
user_data.loc[list(np.argsort(X[1])[::-1][:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
1,Pirates of the Caribbean: At World's End,Gore Verbinski,169.0,2007.0,USA,Action|Adventure|Fantasy,Johnny Depp|Orlando Bloom|Jack Davenport,Color,goddess|marriage ceremoni|marriage propos|pira...,7.1
200,Pirates of the Caribbean: The Curse of the Bla...,Gore Verbinski,143.0,2003.0,USA,Action|Adventure|Fantasy,Johnny Depp|Orlando Bloom|Jack Davenport,Color,caribbean|curs|governor|pirat|undead,8.1
12,Pirates of the Caribbean: Dead Man's Chest,Gore Verbinski,151.0,2006.0,USA,Action|Adventure|Fantasy,Johnny Depp|Orlando Bloom|Jack Davenport,Color,box office hit|giant squid|heart|liar's dic|mo...,7.3
2051,The Libertine,Laurence Dunmore,114.0,2004.0,UK,Drama|Romance,Johnny Depp|Jack Davenport|Richard Coyle,Color,ambassador|charles ii|earl|mistress|poet,6.4
178,Rango,Gore Verbinski,107.0,2011.0,USA,Adventure|Animation|Comedy|Family|Western,Johnny Depp|Ray Winstone|Stephen Root,Color,chameleon|lizard|sheriff|water|western town,7.2


In [34]:
user_data.loc[list(np.argsort(X[9])[::-1][:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
9,Batman v Superman: Dawn of Justice,Zack Snyder,183.0,2016.0,USA,Action|Adventure|Sci-Fi,Henry Cavill|Lauren Cohan|Alan D. Purwin,Color,based on comic book|batman|sequel to a reboot|...,6.9
14,Man of Steel,Zack Snyder,143.0,2013.0,USA,Action|Adventure|Fantasy|Sci-Fi,Henry Cavill|Christopher Meloni|Harry Lennix,Color,based on comic book|british actor playing amer...,7.2
1195,The Count of Monte Cristo,Kevin Reynolds,131.0,2002.0,UK,Action|Adventure|Drama|Romance|Thriller,Henry Cavill|James Frain|Michael Wincott,Color,count|escap|island|reveng|sailor,7.7
2244,The Cold Light of Day,Mabrouk El Mechri,93.0,2012.0,USA,Action|Thriller,Henry Cavill|Bruce Willis|Óscar Jaenada,Color,driving a car down stair|duct tape gag|held ca...,4.9
549,Sucker Punch,Zack Snyder,128.0,2011.0,USA,Action|Fantasy,Jon Hamm|Abbie Cornish|Scott Glenn,Color,alternate r|escap|girl gang|prostitut|samurai,6.1


In [35]:
user_data.loc[list(np.argsort(X[11])[::-1][:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
11,Quantum of Solace,Marc Forster,106.0,2008.0,UK,Action|Adventure,Giancarlo Giannini|Mathieu Amalric|Rory Kinnear,Color,action hero|attempted rap|bond girl|official j...,6.7
530,Munich,Steven Spielberg,163.0,2005.0,France,Drama|History|Thriller,Ayelet Zurer|Moritz Bleibtreu|Mathieu Amalric,Color,arab|black septemb|israel|jew|munich olymp,7.6
4528,Everything Put Together,Marc Forster,87.0,2000.0,USA,Drama,Radha Mitchell|Alan Ruck|Megan Mullally,Color,group of friend|mental breakdown|mental inst|m...,6.6
2186,The Kite Runner,Marc Forster,128.0,2007.0,USA,Drama,Mustafa Haidari|Shaun Toub|Khalid Abdalla,Color,afghanistan|based on novel|boy|friend|kite,7.6
45,World War Z,Marc Forster,123.0,2013.0,USA,Action|Adventure|Horror|Sci-Fi|Thriller,Peter Capaldi|Brad Pitt|Mireille Enos,Color,chao|diseas|infect|pandem|zombi,7.0


In [36]:
user_data.loc[list(np.argsort(X[25])[::-1][:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
25,Titanic,James Cameron,194.0,1997.0,USA,Drama|Romance,Leonardo DiCaprio|Kate Winslet|Gloria Stuart,Color,artist|love|ship|titan|wet,7.7
589,The Abyss,James Cameron,171.0,1989.0,USA,Adventure|Drama|Sci-Fi|Thriller,Michael Biehn|Todd Graff|Mary Elizabeth Mastra...,Color,alien|estranged coupl|ocean|submarin|underwat,7.6
0,Avatar,James Cameron,178.0,2009.0,USA,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder|Joel David Moore|Wes Studi,Color,avatar|futur|marin|nativ|parapleg,7.9
3430,The Terminator,James Cameron,107.0,1984.0,UK,Action|Sci-Fi,Michael Biehn|Brian Thompson|Paul Winfield,Color,cyborg|future war|robot|shootout in a police s...,8.1
1083,Revolutionary Road,Sam Mendes,119.0,2008.0,USA,Drama|Romance,Leonardo DiCaprio|Kate Winslet|Joe Komara,Color,based on novel|children|connecticut|suburb|work,7.3


Les résultats se sont globalement améliorés : 

Les recommandations pour Pirates des Caraïbes sont désormais très satisfaisantes puisque l'on retrouve des films avec Johnny Depp ou Jack Davenport.

Les recommandations pour Superman sont peut être les celles à décevoir un peu. Si avant elles étaient pertinentes et basées sur l'univers fantastiques elles semblent désormais uniquement corrélées à la présence de Zack Snyder, Henry Cavill.

Concernant Quantum of Solace on retrouve bien des films où figure Mathieu Amalric ou produit par Marc Foster.

Enfin Titanic se retrouve associé à des films de James Cameron ce qui est tout à fait logique bien que l'on ait perdu la notion de drame/romance.

Essayons donc de généraliser notre approche de la ressemblance avec les mots clefs et le genre afin que la seule présence d'acteur ou de directeur ne prenne pas le pas sur tout le reste et voyons si cela nous permet d'améliorer nos résultats.

### KNN version 4

In [37]:
def distance(x, y):
    x_booleans = x[4:]
    y_booleans = y[4:]
    z_booleans = x_booleans[(x_booleans-y_booleans == 0) & (x_booleans > 0)]
    x = x[:4]
    y = y[:4]
    z = (x - y)[x - y != 0]
    return sum(z_booleans) - math.sqrt(sum((z)**2))

In [38]:
dist = DistanceMetric.get_metric('pyfunc', func=distance)
X = dist.pairwise(X_scaled)

In [39]:
user_data.loc[list(np.argsort(X[1])[::-1][:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
1,Pirates of the Caribbean: At World's End,Gore Verbinski,169.0,2007.0,USA,Action|Adventure|Fantasy,Johnny Depp|Orlando Bloom|Jack Davenport,Color,goddess|marriage ceremoni|marriage propos|pira...,7.1
200,Pirates of the Caribbean: The Curse of the Bla...,Gore Verbinski,143.0,2003.0,USA,Action|Adventure|Fantasy,Johnny Depp|Orlando Bloom|Jack Davenport,Color,caribbean|curs|governor|pirat|undead,8.1
12,Pirates of the Caribbean: Dead Man's Chest,Gore Verbinski,151.0,2006.0,USA,Action|Adventure|Fantasy,Johnny Depp|Orlando Bloom|Jack Davenport,Color,box office hit|giant squid|heart|liar's dic|mo...,7.3
2051,The Libertine,Laurence Dunmore,114.0,2004.0,UK,Drama|Romance,Johnny Depp|Jack Davenport|Richard Coyle,Color,ambassador|charles ii|earl|mistress|poet,6.4
13,The Lone Ranger,Gore Verbinski,150.0,2013.0,USA,Action|Adventure|Western,Johnny Depp|Ruth Wilson|Tom Wilkinson,Color,hors|outlaw|texa|texas rang|train,6.5


In [40]:
user_data.loc[list(np.argsort(X[9])[::-1][:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
9,Batman v Superman: Dawn of Justice,Zack Snyder,183.0,2016.0,USA,Action|Adventure|Sci-Fi,Henry Cavill|Lauren Cohan|Alan D. Purwin,Color,based on comic book|batman|sequel to a reboot|...,6.9
14,Man of Steel,Zack Snyder,143.0,2013.0,USA,Action|Adventure|Fantasy|Sci-Fi,Henry Cavill|Christopher Meloni|Harry Lennix,Color,based on comic book|british actor playing amer...,7.2
7,Avengers: Age of Ultron,Joss Whedon,141.0,2015.0,USA,Action|Adventure|Sci-Fi,Chris Hemsworth|Robert Downey Jr.|Scarlett Joh...,Color,artificial intellig|based on comic book|captai...,7.5
26,Captain America: Civil War,Anthony Russo,147.0,2016.0,USA,Action|Adventure|Sci-Fi,Robert Downey Jr.|Scarlett Johansson|Chris Evans,Color,based on comic book|knife|marvel cinematic uni...,8.2
872,Superman II,Richard Lester,116.0,1980.0,USA,Action|Adventure|Romance|Sci-Fi,Margot Kidder|Ned Beatty|Jackie Cooper,Color,alien invas|based on comic book|flying superhe...,6.8


In [41]:
user_data.loc[list(np.argsort(X[11])[::-1][:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
11,Quantum of Solace,Marc Forster,106.0,2008.0,UK,Action|Adventure,Giancarlo Giannini|Mathieu Amalric|Rory Kinnear,Color,action hero|attempted rap|bond girl|official j...,6.7
2698,The Diving Bell and the Butterfly,Julian Schnabel,112.0,2007.0,France,Biography|Drama,Emmanuelle Seigner|Mathieu Amalric|Isaach De B...,Color,arc de triomphe pari|based on autobiographi|co...,8.0
2708,Wild Grass,Alain Resnais,104.0,2009.0,France,Drama|Romance,Mathieu Amalric|Sara Forestier|Edouard Baer,Color,dentist|letter|pilot|skateboard|wallet,6.3
4249,The Blue Room,Mathieu Amalric,76.0,2014.0,France,Crime|Romance|Thriller,Mathieu Amalric|Léa Drucker|Laurent Poitrenaux,Color,male pubic hair|manipulative person|murder of ...,6.3
45,World War Z,Marc Forster,123.0,2013.0,USA,Action|Adventure|Horror|Sci-Fi|Thriller,Peter Capaldi|Brad Pitt|Mireille Enos,Color,chao|diseas|infect|pandem|zombi,7.0


In [42]:
user_data.loc[list(np.argsort(X[25])[::-1][:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
25,Titanic,James Cameron,194.0,1997.0,USA,Drama|Romance,Leonardo DiCaprio|Kate Winslet|Gloria Stuart,Color,artist|love|ship|titan|wet,7.7
1083,Revolutionary Road,Sam Mendes,119.0,2008.0,USA,Drama|Romance,Leonardo DiCaprio|Kate Winslet|Joe Komara,Color,based on novel|children|connecticut|suburb|work,7.3
589,The Abyss,James Cameron,171.0,1989.0,USA,Adventure|Drama|Sci-Fi|Thriller,Michael Biehn|Todd Graff|Mary Elizabeth Mastra...,Color,alien|estranged coupl|ocean|submarin|underwat,7.6
0,Avatar,James Cameron,178.0,2009.0,USA,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder|Joel David Moore|Wes Studi,Color,avatar|futur|marin|nativ|parapleg,7.9
281,Terminator 2: Judgment Day,James Cameron,153.0,1991.0,USA,Action|Sci-Fi,Joe Morton|Jenette Goldstein|S. Epatha Merkerson,Color,futur|liquid met|multiple cameo|sexy woman|tim...,8.5


Les résultats se sont améliorés comme escomptés pour Superman et Titanic et sont désormais très pertinents.

Testons deux derniers films d'un style différent pour voir ce qu'il en ressort et valider pleinement notre algorithme. Nous allons tester un film Indien où la religion est très présente, ainsi qu'un film d'animation Mexicain.

In [43]:
user_data.loc[list(np.argsort(X[1025])[::-1][:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
1025,Earth,Deepa Mehta,110.0,1998.0,India,Drama|Romance|War,Nandita Das|Gulshan Grover|Eric Peterson,Color,friend|hindu|india|muslim|sikh,7.8
3633,"Gandhi, My Father",Feroz Abbas Khan,136.0,2007.0,India,Biography|Drama|History,Akshaye Khanna|Bhoomika Chawla|Darshan Jariwala,Color,africa|hindu|india|muslim|south africa,7.4
3132,Partition,Vic Sarin,116.0,2007.0,Canada,Drama|Romance,Jimi Mistry|Jesse Moss|John Light,Color,india|love|muslim|pakistan|sikh,7.0
3219,My Name Is Khan,Karan Johar,128.0,2010.0,India,Adventure|Drama|Thriller,Shah Rukh Khan|Jimmy Shergill|Christopher B. D...,Color,airport|asperger's syndrom|autism|muslim|racia...,8.0
3532,Rang De Basanti,Rakeysh Omprakash Mehra,157.0,2006.0,India,Comedy|Drama|History|Romance,Anupam Kher|Steven Mackintosh|Madhavan,Color,freedom|friend|india|student|urin,8.4


In [44]:
user_data.loc[list(np.argsort(X[494])[::-1][:5]),:]

,movie_title,director_name,duration,title_year,country,genres,actors,colors,plot_keywords,imdb_score
494,Top Cat Begins,Andrés Couturier,89.0,2015.0,Mexico,Animation|Family,Sariann Monaco|David Hoffman|Ben Diskin,Color,NaN,4.4
3929,Compadres,Enrique Begne,101.0,2016.0,Mexico,Action|Comedy,Kevin Pollak|Héctor Jiménez|Erick Elias,Color,NaN,5.0
3679,Treading Water,Analeine Cal y Mayor,92.0,2013.0,Mexico,Comedy|Drama,Zoë Kravitz|Douglas Smith|Kim Ly,Color,NaN,6.2
4385,Heli,Amat Escalante,105.0,2013.0,Mexico,Crime|Drama|Romance,Kenny Johnston|Andrea Vergara|Gabriel Reyes,Color,corrupt polic|drug cartel|drug|mexico|polic,6.8
3111,"Cinco de Mayo, La Batalla",Rafa Lara,125.0,2013.0,Mexico,Drama|History|War,Jorge Luis Moreno|Andrés Montiel|William Miller,Color,NaN,6.2


Concernant le film Indien les résultats sont très satisfaisants avec une bonne présence de films Indiens d'amour où la religion est mêlée. Pour le film d'animation Mexicain on se rend compte que c'est plus le Mexique que le côté animation qui ressort du fait du faible nombre de films Mexicains dans notre dataset. Toutefois cela reste satisfaisant.

Il s'agit donc de la version finale de notre modèle. Il ne reste désormais plus qu'à exporter sous csv ce dont nous aurons besoin pour notre API : la matrice de distance et la dataframe user_data qui sera utilisé pour afficher le contenu destiné à l'utilisateur.

Commençons par sauver notre matrice sous csv. Plutôt que de sauver l'intégralité de notre matrice, nous n'avons besoin que de sauvegarder les index des 15 voisins les plus proches.

In [48]:
neighbours_matrix = []
for i in range(len(X) - 1):
    neighbours_matrix.append(list(np.argsort(X[i])[::-1][:16][1:]))

In [49]:
pd.DataFrame(neighbours_matrix).to_csv("matrix.csv", encoding="utf-8")

In [50]:
user_data.to_csv("user_data.csv", columns=user_data.columns, encoding="utf-8")